# CSC/XMM validation catalogs, XMM/CSC hostless catalog

In [32]:
import sys
sys.path.append('../')

In [33]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import matplotlib.pyplot as plt
# plt.style.use('seaborn')

import seaborn as sns

import json

from scripts.utils import set_mpl, data_path
from scripts.cross_match_scripts import cross_match_data_frames, fits_to_pandas, add_separation_columns
set_mpl()

%matplotlib inline
from astropy.table import Table

matplotlib settings set


In [34]:
def csc_error_converter(df: pd.DataFrame,
                        r0_colname='err_ellipse_r0',
                        r1_colname='err_ellipse_r1') -> pd.DataFrame:
    """
    The function converts default radii `r0_colname`
    and `r1_colname` to the one-sigma error.

    Args:
        df (pd.DataFrame): DataFrame with `r0_colname` and
        `r1_colname` columns.
        
        r0_colname (str): major radius of the 95% confidence
        level position error ellipse.
        Defaults to 'err_ellipse_r0'.
        
        r1_colname (str): minor radius of the 95% confidence
        level position error ellipse.
        Defaults to 'err_ellipse_r1'.

    Returns:
        pd.DataFrame: one-sigma error in arcseconds.
    """

    # Conversion coefficient
    csc_sigma_coeff = np.sqrt(-(1 / (2 * np.log(1 - .95))))

    err_r1 = df[r0_colname]
    err_r2 = df[r1_colname]
    # Effective error
    csc_err_eff = np.sqrt(err_r1 ** 2 + err_r2 ** 2)

    csc_1sigma = csc_sigma_coeff * csc_err_eff

    return csc_1sigma


def vot2pd_csc(csc_cat_path: str,
               colnames: list,
               save_coords: bool = False, 
               radec_fits_name: str = 'cscresults_name_radec') -> pd.DataFrame:
    """
    The function converts the votable file to pandas DataFrame.

    Optionally saves the separate file with the coordinates and names of
    the CSC sources to the FITS file.

    Args:
        csc_cat_path (str): the path to the votable file.
        colnames (list): column names of the votable file.
        save_coords (bool): if True, saves the separate file with the coordinates
        and names of the CSC sources to the FITS file.
        radec_fits_name (str): name of the FITS file with
        the coordinates and names of the CSC sources to be saved.

    Returns:
        pd.DataFrame: converted catalogue.
    """
    
    vot_table = Table.read(csc_cat_path, format='votable')
    
    df = vot_table.to_pandas()
    df.columns = colnames

    df = df.assign(flux_csc_05_2 = lambda x: x.flux_aper_s + x.flux_aper_m)

    one_sigma_errors = csc_error_converter(df)

    df = df.assign(radec_err_csc = one_sigma_errors)

    df['r_98_csc'] = tsource_r(sigma=df['radec_err_csc'], t_thresh=.02)

    if save_coords:

        df_init_coords = df[['name', 'ra', 'dec']]
        Table.from_pandas(df_init_coords).write(f'data/{radec_fits_name}.fits', format='fits')

    return df


def tsource_r(sigma: float, t_thresh: float) -> float:
    """
    Calculates the radius of circle which contains the probability `t_thresh`
    NOT to find a counterpart for a source with localization error `sigma`.

    Args:
        sigma (float): localization error in arcsec.
        t_thresh (float): probability to NOT find a counterpart.

    Returns:
        float: radius of circle in arcsec.
    """
    
    rsearch = sigma * np.sqrt(-2 * np.log(t_thresh))
    
    return rsearch


def fsource_r(rho: float, f_thresh: float) -> float:
    """
    The function calculates the radius which corresponds to the probability `f_thresh`
    to FIND one or more false sources. 

    Args:
        rho (float): the density of the false sources (in arcmin^{-2}).
        f_thresh (float): probability to FIND one or more false sources.

    Returns:
        float: radius (in arcsec).
    """

    rho = rho / 3600 # arcmin^{-2} -> arcsec^{-2}
    pf_r = np.sqrt(-1 / (rho * np.pi) * np.log(1 - f_thresh))
    
    return pf_r


def poserr2sigma_coeff(conf_level: float) -> float:
    """
    Calculates convertion coefficient to go from
    positional error to sigma given confidence level.

    For details see:
    https://www.notion.so/Theory-ca6e7795b40c43b4ba6d96bc59727efa#b4d9fc11ff8243a3834e9eeba08c2273
    """
    coeff = (-2 * np.log(1 - conf_level)) ** -0.5
    
    return coeff


def pos_r(sigma: float, conf_level: float) -> float:
    """
    Calculates the radius of circle which contains the probability `conf_level`
    of finding (opposite to tsource_r()) a counterpart for a source with
    localization error `sigma`.

    For details see:
    https://www.notion.so/Theory-ca6e7795b40c43b4ba6d96bc59727efa#0ba88df64d2d4d9583f93d63dbe1b927

    Args:
        sigma (float): localization error in arcsec.
        conf_level (float): probability of finding a counterpart.

    Returns:
        float: radius of circle in arcsec.
    """
    
    r_pos = sigma * np.sqrt(-2 * np.log(1 - conf_level))

    return r_pos


# def only_reliable_xmm(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Filters out unreliable XMM sources.
#     """
#     reliable_df = df[
#                     ((df['xmm_SC_SUM_FLAG'] == 0) |
#                     (df['xmm_SC_SUM_FLAG'] == 1)) &
#                     (df['xmm_SC_DET_ML'] > 10) &
#                     ~(df['xmm_SC_VAR_FLAG'] == True) &
#                     (df['xmm_SC_EXTENT'] == 0) &
#                     (df['xmm_CONFUSED'] == False)
#                     ]

#     return reliable_df


def only_reliable_xmm(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filters out unreliable XMM sources.
    """
    reliable_df = df[
                    ((df['SC_SUM_FLAG'] == 0) |
                    (df['SC_SUM_FLAG'] == 1)) &
                    (df['SC_DET_ML'] > 10) &
                    ~(df['SC_VAR_FLAG'] == True) &
                    (df['SC_EXTENT'] == 0) &
                    (df['CONFUSED'] == False)
                    ]

    return reliable_df




def xray_filtration(df: pd.DataFrame,
                    DL_thresh: float = 6,
                    EL_thresh: float = 6,
                    verbouse=True) -> pd.DataFrame:
    """
    Filters X-ray sources.
    TODO: remake processing of duplicates
    """
    
    if verbouse:
        print(f'DET_LIKE_0 > {DL_thresh}')
        print(f'EXT_LIKE < {EL_thresh}')
        print()

        print(f'Before X-ray source filters: {len(df)}')

    df = df[(df['DET_LIKE_0'] > DL_thresh)&
            (df['EXT_LIKE'] < EL_thresh)]

    if verbouse:
        print(f'After X-ray source filters: {len(df)}')
        print()


    # Manually get rid of faint sources in duplicated pairs
    df = df[~((df['srcname_fin']=='SRGe J104659.3+573056')&(df['DET_LIKE_0'] < 20))]
    df = df[~((df['srcname_fin'] == 'SRGe J104700.7+574558')&(df['DET_LIKE_0'] < 20))]
    print('Weak ERO duplicates removed (temporary measure)')
    print()

    return df



srg_names = {
            'id_src_name': 'srcname_fin',  # Индексы рентгеновских источников
            'x_ra_name': 'RA_fin',  # Координаты рентгеновских источников
            'x_dec_name': 'DEC_fin',
            'dl_name': 'DET_LIKE_0',  # Detection Likelihood
            'x_flux_name': 'flux_05-20',
            'ext_name': 'EXT_LIKE',  # Протяженность рентгеновских источников
            'ls_ra_name': 'ra',  # Координаты источников DESI
            'ls_dec_name': 'dec',
            'r_98_name': 'pos_r98',  # Позиционная ошибка
            'sigma_2d_name': 'pos_sigma_2d'
            }


***

In [35]:
# Каталог ERO (9500)
ero_df = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14.pkl')
ero_df = xray_filtration(ero_df, DL_thresh=6, EL_thresh=6)
ero_df.sample(5)

DET_LIKE_0 > 6
EXT_LIKE < 6

Before X-ray source filters: 9215
After X-ray source filters: 9215

Weak ERO duplicates removed (temporary measure)



,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH
6798,SRGe J104005.9+590048,160.024550,59.013326,149.195964,50.887199,11.977868,11.376833,4.918419e-15,1.444889e-15,23.430573,6.883224,4352.160156,0.0,0.0,0.0,6883,5776,160.026558,59.012036,5.291837,53.553806,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.005384,0.001582,18.912590,160.024550,59.013326,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,0,0,16.080000,15.757000,12.107,21.2,9.4,4.0,1,3,0,1237655368745747224,0,3.907993,58842985387,5.291837,4.282169,6.464329,10.481667,133.924872,45.773271,3.784915e-15,7.880557e+19
8201,SRGe J105505.7+583753,163.773945,58.631291,147.540542,52.578893,21.477022,7.233522,3.817911e-15,1.370261e-15,17.809851,6.392015,4261.691406,0.0,0.0,0.0,7799,6662,163.775953,58.630001,9.573750,133.002457,0.000009,6.268413e+08,6.270701e+08,0.0,0.0,0.004179,0.001500,17.606882,163.773945,58.631291,-1,-1.000000,0,-1,NaN,-1.0,-1,LHW J105506+583749,NaN,Radio,4.689296,1,0,1,0,16.954000,16.462999,12.322,10.7,4.8,0.5,1,1,1,1237658303816794554,0,1.407866,60266250545,9.573750,7.678181,11.590922,18.794243,136.573234,46.512131,2.938031e-15,7.079081e+19
8415,SRGe J103607.7+552334,159.031949,55.392665,154.361228,52.674578,10.768907,7.908237,3.675064e-15,1.318911e-15,15.732603,5.646132,3910.958496,0.0,0.0,0.0,9192,8037,159.033957,55.391376,4.742928,100.592934,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.004023,0.001444,16.484438,159.031949,55.392665,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,72953551745,4.742928,3.849957,5.811865,9.423720,135.903939,42.399993,2.828105e-15,6.797224e+19
5700,SRGe J105355.8+592320,163.482489,59.388767,146.834458,51.938962,10.529666,17.172867,5.992850e-15,1.507361e-15,30.492296,7.669623,4648.408203,0.0,0.0,0.0,6075,5021,163.484498,59.387477,4.634086,111.971382,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.006560,0.001650,19.172325,163.482489,59.388767,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,0,15.442000,14.649000,11.961,26.4,20.6,3.8,2,1,0,1237658304890536797,0,4.551751,57459112900,4.634086,3.764427,5.682749,9.214364,135.774689,47.057395,4.611731e-15,7.378133e+19
1183,SRGe J105619.9+592150,164.083056,59.363798,146.509378,52.171896,4.182891,143.766144,2.293068e-14,2.590075e-15,109.363945,12.352920,4357.172363,0.0,0.0,0.0,1014,823,164.085065,59.362508,1.664787,60.520733,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.025100,0.002835,19.952257,164.083056,59.363798,860599192853440256,6.415124,1,0,20.328367,-1.0,0,SDSS J105619.91+592150.2,1.050561,QSO,6.411523,1,0,0,2,16.106001,15.262000,12.039,20.7,13.4,3.7,2,1,1,1237655109447123135,9214519993328488448,6.466043,57551357911,1.664787,1.495412,2.257462,3.660390,136.167425,47.208493,1.764605e-14,6.480471e+19


In [36]:
desi = pd.read_pickle(data_path+'desi_lh.gz_pkl', compression='gzip')
desi.sample(5)

,release,objid,brickid,ra,dec,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,type,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,ref_cat,ref_id,mjd_max,mjd_min,iso_max,iso_min,desi_id
1751809,9011,40,604374,160.983357,55.656080,0.078155,-0.099308,0.513881,1.640983,-1.293175,8.780919,454.924260,523.00510,154.27094,157.431380,4.594570,1.084449,0.001372,0.000016,25.249178,NaN,23.215899,21.961185,NaN,20.141012,15.855100,1.787344,-1.233467,6.447752,3.517437,-1.346672,0.325204,1.845809,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57902.156667,57464.298032,2017-05-29 03:45:36.000,2016-03-17 07:09:10.000,9011_604374_40
1185688,9011,886,611481,156.624307,58.038087,0.574662,1.269508,1.508239,0.619833,0.873703,-14.460260,-103.746130,404.76596,135.42163,51.561924,4.405509,1.061682,0.001407,0.000017,23.071003,22.220388,22.042345,23.017570,22.645520,NaN,NaN,11.561502,14.773368,10.830155,1.300986,0.900246,-0.542359,-0.430739,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.391325,57819.340926,2018-03-17 09:23:30.460,2017-03-07 08:10:56.000,9011_611481_886
399091,9011,2853,617450,158.512035,60.023848,0.207647,0.354553,0.587514,0.105130,2.250607,-4.877312,54.886806,506.24606,208.20728,72.319400,4.647777,1.069372,0.001064,0.000013,24.177320,23.606018,23.066390,24.944004,21.618220,NaN,18.151247,4.672049,5.115981,4.996270,0.226646,2.327362,-0.159093,0.197443,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.411692,57496.218750,2018-03-17 09:52:50.192,2016-04-18 05:15:00.000,9011_617450_2853
729347,9011,2379,614522,164.437496,58.997761,0.566755,0.605528,0.910377,1.605219,1.277122,-28.349882,-16.486801,422.67370,132.64700,59.547560,4.571750,1.065419,0.001076,0.000014,23.087063,23.024826,22.590850,21.984478,22.233383,NaN,NaN,11.651923,6.974015,7.025112,3.432224,1.318235,-0.929882,-0.060948,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57916.223611,57837.296689,2017-06-12 05:22:00.000,2017-03-25 07:07:13.949,9011_614522_2379
362530,9011,2603,618175,160.272503,60.269324,0.206994,1.086254,2.882573,6.363282,0.724000,-52.774986,4.669880,715.27216,332.94763,111.530754,4.199637,1.090895,0.001019,0.000013,24.185177,22.393381,21.341158,20.489370,22.849777,NaN,20.826666,5.535975,19.820715,30.442314,13.040285,0.756188,-1.685067,0.016699,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58226.271131,57414.406030,2018-04-18 06:30:25.741,2016-01-27 09:44:41.000,9011_618175_2603


# CSC

In [37]:
# CSC table conversion
csc_columns = ['name', 'ra', 'dec', 'err_ellipse_r0', 'err_ellipse_r1', 'err_ellipse_ang',
               'significance', 'likelihood', 'likelihood_class', 'conf_flag', 'dither_warning_flag',
               'extent_flag', 'pileup_flag', 'sat_src_flag', 'streak_src_flag', 'var_flag',
               'flux_aper_s', 'flux_aper_lolim_s', 'flux_aper_hilim_s', 'flux_aper_m',
               'flux_aper_lolim_m', 'flux_aper_hilim_m']

# Filtration
# data/cscresults.vot obtained via CSCview software
csc_init_df = vot2pd_csc(csc_cat_path=data_path+'cscresults.vot', colnames=csc_columns)

# CSC catalogue filtering
csc_df = csc_init_df[(csc_init_df['conf_flag'] == False) &
                              (csc_init_df['extent_flag'] == False) &
                              (csc_init_df['sat_src_flag'] == False) &
                              (csc_init_df['streak_src_flag'] == False) &
                              (csc_init_df['pileup_flag'] == False) &
                              (csc_init_df['dither_warning_flag'] == False)]


csc_df = csc_df[(csc_df['likelihood'] > 10) &
                                  (csc_df['likelihood'] < 10 ** 10)]


csc_df = csc_df.assign(flux_05_2 = lambda x: x.flux_aper_s + x.flux_aper_m)

# Ошибки на поток 0.5-2 кэВ
s_up = csc_df.flux_aper_hilim_s - csc_df.flux_aper_s
s_down = csc_df.flux_aper_s - csc_df.flux_aper_lolim_s
s_err = np.sqrt(s_up ** 2 + s_down ** 2)
m_up = csc_df.flux_aper_hilim_m - csc_df.flux_aper_m
m_down = csc_df.flux_aper_m - csc_df.flux_aper_lolim_m
m_err = np.sqrt(m_up ** 2 + m_down ** 2)

sm_err = np.sqrt(s_err ** 2 + m_err ** 2).values
csc_df['flux_aper_sm_err'] = sm_err


print(f'CSC Sources: {csc_df.shape[0]}')
csc_df.sample(5)

CSC Sources: 267265


,name,ra,dec,err_ellipse_r0,err_ellipse_r1,err_ellipse_ang,significance,likelihood,likelihood_class,conf_flag,dither_warning_flag,extent_flag,pileup_flag,sat_src_flag,streak_src_flag,var_flag,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,flux_aper_m,flux_aper_lolim_m,flux_aper_hilim_m,flux_csc_05_2,radec_err_csc,r_98_csc,flux_05_2,flux_aper_sm_err
181716,2CXO J135940.6-193735,209.919415,-19.626469,5.292648,3.422285,123.436500,3.272727,26.609381,TRUE,False,False,False,False,False,False,False,1.714139e-15,6.049901e-16,2.772871e-15,0.000000e+00,0.000000e+00,4.016454e-16,1.714139e-15,2.574903,7.202388,1.714139e-15,1.585069e-15
115870,2CXO J103845.4+050610,159.689192,5.102939,4.247489,2.538897,161.764346,3.028571,18.593101,TRUE,False,False,False,False,False,False,False,0.000000e+00,0.000000e+00,1.342482e-15,1.628838e-15,5.235551e-16,2.734121e-15,1.628838e-15,2.021635,5.654815,1.628838e-15,2.060475e-15
97250,2CXO J091714.9+690604,139.312458,69.101194,2.478313,2.021158,63.343642,7.875053,118.670110,TRUE,False,False,False,False,False,False,True,7.783336e-15,5.578058e-15,9.858893e-15,9.107087e-15,7.467279e-15,1.074690e-14,1.689042e-14,1.306502,3.654481,1.689042e-14,3.814332e-15
221469,2CXO J164658.8-455146,251.745366,-45.862816,0.778013,0.740384,12.890549,3.176471,37.647291,TRUE,False,False,False,False,False,False,False,NaN,NaN,NaN,5.472253e-16,3.054281e-16,7.890225e-16,NaN,0.438770,1.227306,NaN,NaN
32832,2CXO J023303.8+612807,38.265959,61.468677,1.287235,0.918479,1.816250,2.378378,15.884785,MARGINAL,False,False,False,False,False,False,False,0.000000e+00,0.000000e+00,2.456663e-16,4.311699e-16,2.321684e-16,6.301714e-16,4.311699e-16,0.646031,1.807046,4.311699e-16,3.735708e-16


## CSC x ERO cross-match

In [38]:
ero_csc = cross_match_data_frames(ero_df, csc_df,
 'RA_fin', 'DEC_fin', 'ra', 'dec', match_radius = 30, df_prefix = 'csc')
ero_csc = ero_csc.query("csc_n_near==1 & csc_n_matches==1")
print('cross-matches: only one csc within 30'' and it is unique', ero_csc.shape[0])
assert len(ero_csc) == ero_csc.csc_name.nunique()

ero_csc.sample(5)

cross-match radius 30 arcsec
total matches: 739 out of 9215 x 267265
	 total unique pairs: 692
	 total non-unique pairs (duplicates in df2): 47
cross-matches: only one csc within 30 and it is unique 566


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_flux_csc_05_2,csc_radec_err_csc,csc_r_98_csc,csc_flux_05_2,csc_flux_aper_sm_err,csc_sep,csc_n_near,csc_n_matches
408,SRGe J104452.6+560241,161.219160,56.044655,152.266558,53.267469,8.114606,16.974001,7.134705e-15,1.781543e-15,33.583111,8.385739,4300.237305,0.0,0.0,0.0,1195,957,161.221169,56.043365,3.529299,34.172497,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.007810,0.001950,20.931793,161.219160,56.044655,-1,-1.000000,0,-1,NaN,-1.0,-1,SDSS J104451.99+560246.8,2.6285,QSO,14.091835,1,1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,70310960881,3.529299,2.901027,4.379367,7.100979,136.895339,43.590306,5.490434e-15,5.600098e+19,2CXO J104452.0+560246,161.216751,56.046232,3.193581,2.393824,179.998861,3.085714,20.057914,TRUE,False,False,False,False,False,False,False,2.271756e-15,8.437950e-16,3.699717e-15,1.715249e-15,6.968197e-16,2.680076e-15,3.987004e-15,1.630544,4.560876,3.987004e-15,2.458909e-15,7.465400,1,1
649,SRGe J104722.9+585033,161.845225,58.842524,148.412011,51.706862,10.014643,7.975955,4.093821e-15,1.396039e-15,19.089035,6.509577,4259.931641,0.0,0.0,0.0,8904,7750,161.847234,58.841235,4.399482,90.190491,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.004481,0.001528,19.789173,161.845225,58.842524,-1,-1.000000,0,-1,NaN,-1.0,-1,CLANS 588,0.6800,Galaxy,8.042606,3,1,0,0,16.320999,16.488001,12.455,16.5,5.2,0.7,1,1,0,1237655109446598967,0,7.787251,59477280415,4.399482,3.580303,5.404797,8.763675,135.194666,46.137733,3.150355e-15,8.375269e+19,2CXO J104722.5+585033,161.843914,58.842556,0.885940,0.814588,99.070735,9.369712,280.821016,TRUE,False,False,False,False,False,False,False,5.103785e-15,4.083028e-15,6.067833e-15,3.489494e-15,2.877300e-15,4.101687e-15,8.593278e-15,0.491682,1.375308,8.593278e-15,1.649514e-15,2.444302,1,1
571,SRGe J104605.8+583909,161.524349,58.652389,148.819802,51.710246,10.471605,12.275084,5.067076e-15,1.432778e-15,23.729176,6.709717,4278.314453,0.0,0.0,0.0,5945,4905,161.526358,58.651099,4.607659,88.625359,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.005546,0.001568,17.571255,161.524349,58.652389,855289449340333696,3.572114,2,1,17.631144,NaN,0,SDSS J104606.35+583907.8,NaN,**,3.878739,1,1,0,0,14.328000,14.149000,12.571,39.9,27.1,2.3,1,2,1,1237658304353140786,7988356707222048768,3.854136,60187115638,4.607659,3.743670,5.651415,9.163556,135.142357,45.887643,3.899313e-15,7.655071e+19,2CXO J104606.3+583906,161.526463,58.651809,2.285903,1.758722,178.861196,3.937500,42.452823,TRUE,False,False,False,False,False,False,False,3.230448e-15,2.283593e-15,4.121606e-15,8.725464e-16,5.198149e-16,1.206713e-15,4.102994e-15,1.178297,3.295874,4.102994e-15,1.388087e-15,4.474908,1,1
634,SRGe J105320.4+574011,163.334938,57.669679,148.953839,53.078122,9.322067,8.519584,4.217671e-15,1.406030e-15,20.002108,6.668030,4332.619141,0.0,0.0,0.0,9136,7981,163.336946,57.668389,4.083262,105.451553,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.004617,0.001539,20.410501,163.334938,57.669679,-1,-1.000000,0,-1,NaN,-1.0,-1,[BCH2008] 572,NaN,X,11.261499,1,1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,63918107779,4.083262,3.332702,5.031022,8.157612,137.056044,45.581503,3.245663

## CSC x DESI cross-match

In [39]:
csc_desi = cross_match_data_frames(ero_csc, desi, colname_ra1 = 'csc_ra',colname_dec1 = 'csc_dec' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')

cross-match radius 15 arcsec
total matches: 2543 out of 566 x 2418574
	 total unique pairs: 2543
	 total non-unique pairs (duplicates in df2): 0


In [40]:
csc_desi_closest = (csc_desi
    .loc[csc_desi.groupby('csc_name')['desi_sep'].idxmin()]
    )

#csc_hostless = csc_desi_closest.query("desi_sep>2*csc_r_98_csc & desi_sep>2*1.43")
csc_hostless = csc_desi_closest.query("desi_sep>2*csc_r_98_csc")
csc_hostless = csc_hostless.query('csc_sep<20')
csc_hostless['csc_ero_flux_ratio'] = csc_hostless['csc_flux_csc_05_2'] / csc_hostless['flux_05-20']

print('hostless csc near chandra sources', csc_hostless.shape[0])
csc_hostless = csc_hostless[['srcname_fin',	'RA_fin',	'DEC_fin', 'pos_r98', 'flux_05-20', 'csc_name', 'csc_ra',	'csc_dec', 'csc_r_98_csc', 'csc_flux_05_2', 'csc_sep', 'desi_desi_id', 'desi_ra', 'desi_dec', 'desi_sep', 'csc_ero_flux_ratio']]

csc_hostless.rename(columns = {'csc_sep':'ero_csc_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

csc_hostless = add_separation_columns(csc_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert csc_hostless.srcname_fin.nunique()==csc_hostless.csc_name.nunique()
assert csc_hostless.srcname_fin.nunique()==csc_hostless.shape[0]
csc_hostless.sample(7)

hostless csc near chandra sources 29


,srcname_fin,RA_fin,DEC_fin,pos_r98,flux_05-20,csc_name,csc_ra,csc_dec,csc_r_98_csc,csc_flux_05_2,ero_csc_sep,desi_desi_id,desi_ra,desi_dec,desi_sep_minimal,csc_ero_flux_ratio,desi_ero_sep
2527,SRGe J104917.5+585133,162.322873,58.859156,14.623010,2.516668e-15,2CXO J104916.5+585133,162.319141,58.859426,2.428205,3.517709e-16,7.014423,9011_613770_553,162.318576,58.856306,11.279802,0.139776,13.008652
1381,SRGe J104548.9+590827,161.453772,59.140960,6.432184,5.889830e-15,2CXO J104549.0+590830,161.454349,59.141748,2.506897,0.000000e+00,3.027328,9011_615259_3071,161.452012,59.140319,6.714390,0.000000,3.988206
2489,SRGe J104058.8+593458,160.245165,59.582793,10.160961,2.596939e-15,2CXO J104058.4+593455,160.243655,59.581978,1.577766,9.499923e-15,4.022084,9011_615994_2615,160.243453,59.583099,4.052236,3.658123,3.309341
2478,SRGe J105323.6+573829,163.348168,57.641255,9.384750,2.619025e-15,2CXO J105324.2+573828,163.350964,57.641244,1.679285,NaN,5.388710,9011_610726_1110,163.348859,57.640138,5.684306,NaN,4.234390
2344,SRGe J104410.7+585421,161.044430,58.905714,10.203106,3.030344e-15,2CXO J104411.1+585424,161.046365,58.906818,1.161780,6.099061e-15,5.361330,9011_614515_2457,161.045700,58.906094,2.884135,2.012663,2.729407
1575,SRGe J104544.5+585020,161.435576,58.838784,6.675908,5.020989e-15,2CXO J104544.2+585019,161.434203,58.838799,2.049579,4.130239e-15,2.557606,9011_613768_1041,161.428911,58.837392,11.085349,0.822595,13.389330
1363,SRGe J105118.0+552353,162.824877,55.398127,10.753160,6.044226e-15,2CXO J105117.8+552354,162.824260,55.398573,2.010836,3.740731e-15,2.045384,9011_603560_4390,162.820078,55.397071,10.114483,0.618893,10.520464


***

# 4XMM DR10

## XMM x EROSITA (done in topcat)

In [41]:
ero_xmm = pd.read_csv(data_path+'xmm_allsky_full_ero_slim_point_30sec.csv')
ero_xmm = only_reliable_xmm(ero_xmm) #TODO CHECK IF XMM IS RELIABLE

print(f'Reliable XMM Sources within 30 arcsec from eROSITA: {ero_xmm.shape[0]}')

# Поток 0.5-2 кэВ
ero_xmm = ero_xmm.assign(flux_05_2 = lambda x: x.SC_EP_2_FLUX + x.SC_EP_3_FLUX)
# Ошибки на поток 0.5-2 кэВ
ero_xmm = ero_xmm.assign(flux_05_2_err = lambda x: np.sqrt(x.SC_EP_2_FLUX_ERR ** 2 + x.SC_EP_3_FLUX_ERR ** 2))

xmm_err = ero_xmm['SC_POSERR']
xmm_sigma_coeff = poserr2sigma_coeff(.63)
xmm_sigma = xmm_sigma_coeff * xmm_err
ero_xmm.insert(11, 'sigma', xmm_sigma)
# pos_r98
xmm_r98 = pos_r(xmm_sigma, .98)
ero_xmm.insert(11, 'xmm_pos_r98', xmm_r98)


ero_xmm['xmm_ero_flux_ratio'] = ero_xmm['flux_05_2']\
                                                / ero_xmm['flux_05-20']



ero_xmm['GroupSize'] = ero_xmm['GroupSize'].fillna(1)
ero_xmm = ero_xmm[ero_xmm['GroupSize']==1]

print(f'XMM Sources after filters (only one within 30 arcsec): {ero_xmm.shape[0]}')



ero_xmm.sample(10)

Reliable XMM Sources within 30 arcsec from eROSITA: 844
XMM Sources after filters (only one within 30 arcsec): 740


,srcname_fin,RA_fin,DEC_fin,flux_05-20,pos_sigma_2d,pos_r98,DET_LIKE_0,EXT_LIKE,pos_r98_corr,SRCID,IAUNAME,xmm_pos_r98,sigma,SC_RA,SC_DEC,SC_POSERR,SC_DET_ML,SC_EP_1_FLUX,SC_EP_1_FLUX_ERR,SC_EP_2_FLUX,SC_EP_2_FLUX_ERR,SC_EP_3_FLUX,SC_EP_3_FLUX_ERR,SC_EP_4_FLUX,SC_EP_4_FLUX_ERR,SC_EP_5_FLUX,SC_EP_5_FLUX_ERR,SC_EP_8_FLUX,SC_EP_8_FLUX_ERR,SC_EP_9_FLUX,SC_EP_9_FLUX_ERR,SC_HR1,SC_HR1_ERR,SC_HR2,SC_HR2_ERR,SC_HR3,SC_HR3_ERR,SC_HR4,SC_HR4_ERR,SC_EXTENT,SC_EXT_ERR,SC_EXT_ML,SC_CHI2PROB,SC_FVAR,SC_FVARERR,SC_VAR_FLAG,SC_SUM_FLAG,SC_EP_8_FMIN,SC_EP_8_FMIN_ERR,SC_EP_8_FMAX,SC_EP_8_FMAX_ERR,MJD_FIRST,MJD_LAST,N_DETECTIONS,CONFUSED,WEBPAGE_URL,GroupID,GroupSize,Separation,flux_05_2,flux_05_2_err,xmm_ero_flux_ratio
862,SRGe J104710.8+590202,161.794844,59.033926,3.354927e-15,3.165966,8.855680,10.087673,0.0,9.741248,205541201010053,4XMM J104711.7+590203,1.628148,0.582074,161.798838,59.034329,0.820808,60.8720,1.082580e-15,2.647800e-16,1.537760e-15,3.076640e-16,1.946040e-15,4.323310e-16,4.876190e-16,5.234260e-16,6.541300e-16,2.782970e-15,8.344600e-15,3.080100e-15,6.772550e-15,9.379250e-16,-0.094611,0.107965,0.270191,0.107122,-0.764820,0.110608,0.376193,0.259903,0.0,NaN,-0.561316,0.762212,NaN,NaN,False,0,7.892490e-15,3.177080e-15,3.508990e-14,2.479260e-14,54750.880949,54932.883727,3,False,http://xmm-catalog.irap.omp.eu/source/20554120...,NaN,1.0,7.539245,3.483800e-15,5.306291e-16,1.038413
501,SRGe J103233.4+574355,158.139085,57.731908,7.285684e-15,2.452257,6.859330,33.794980,0.0,7.545263,203032602010024,4XMM J103233.6+574354,1.120902,0.400730,158.140144,57.731758,0.565087,261.9020,2.707090e-15,3.325700e-16,4.084160e-15,3.951770e-16,3.441690e-15,4.146130e-16,5.180310e-15,9.065810e-16,4.303070e-15,3.654940e-15,2.151830e-14,3.891860e-15,1.375610e-14,9.574360e-16,0.185317,0.073842,-0.158368,0.073249,-0.294619,0.093186,-0.688371,0.207931,0.0,NaN,-1.478330,0.146056,0.401004,0.267952,False,1,1.968810e-14,4.979400e-15,2.439170e-14,6.239060e-15,53467.914178,53510.606285,2,False,http://xmm-catalog.irap.omp.eu/source/20303260...,NaN,1.0,2.105194,7.525850e-15,5.727729e-16,1.032964
73,SRGe J103953.2+574056,159.971719,57.682305,3.338060e-14,1.119443,3.131249,332.282800,0.0,5.000000,208223909010004,4XMM J103953.0+574055,1.710795,0.611621,159.971041,57.682204,0.862473,331.6260,3.914980e-15,1.504500e-15,1.724190e-14,2.819540e-15,3.751560e-14,4.236590e-15,4.675700e-14,8.693400e-15,4.533300e-14,3.955380e-14,1.623080e-13,4.099060e-14,1.069050e-13,8.632380e-15,0.412685,0.117544,0.334194,0.081943,-0.382069,0.090961,-0.619741,0.242873,0.0,NaN,-1.897860,0.779048,NaN,NaN,False,0,1.623080e-13,4.099060e-14,1.623080e-13,4.099060e-14,58441.381088,58441.542685,1,False,http://xmm-catalog.irap.omp.eu/source/20822390...,NaN,1.0,1.354440,5.475750e-14,5.089057e-15,1.640399
53,SRGe J104403.9+590244,161.016207,59.045454,3.986401e-14,1.166767,3.263622,406.211800,0.0,5.000000,205541201010012,4XMM J104404.0+590242,0.781061,0.279235,161.016991,59.045009,0.393761,862.4070,9.982050e-15,7.091160e-16,1.019530e-14,7.093520e-16,1.423380e-14,9.273980e-16,1.542190e-14,1.804360e-15,4.163470e-14,1.118990e-14,1.007280e-13,1.153460e-14,4.534160e-14,1.958180e-15,-0.061318,0.045152,0.096709,0.045304,-0.328360,0.048894,-0.148237,0.099991,0.0,NaN,-1.269850,0.224664,0.349189,0.337245,False,0,7.551500e-14,2.192560e-14,1.454400e-13,2.125010e-14,54750.880949,54932.883727,4,False,http://xmm-catalog.irap.omp.eu/source/20554120...,NaN,1.0,2.162691,2.442910e-14,1.167582e-15,0.612811
233,SRGe J103200.5+584427,158.002008,58.740872,1.396705e-14,1.848640,5.170923,62.164276,0.0,5.688015,201429701010028,4XMM J103201.0+584422,5.384580,1.925024,158.004215,58.739454,2.714560,21.4282,6.168180e-15,2.832870e-15,9.266210e-15,3.775780e-15,1.098830e-14,4.907860e-15,1.426080e-14,1.167490e-14,0.000000e+00,9.914770e-14,4.068350e-14,1.000650e-13,3.827710e-14,1.093520e-14,0.123147,0.302333,-0.077196,0.300493,-0.396686,0.392918,-1.000000,4.109680,0.0,NaN,-1.106240,NaN,NaN,NaN,NaN,0,4.068350e-14,1.000650e-13,4.068350e-14,1.000650e

In [42]:
xmm_desi = cross_match_data_frames(ero_xmm, desi, colname_ra1 = 'SC_RA',colname_dec1 = 'SC_DEC' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')

cross-match radius 15 arcsec
total matches: 3284 out of 740 x 2418574
	 total unique pairs: 3284
	 total non-unique pairs (duplicates in df2): 0


In [43]:
xmm_desi_closest = (xmm_desi
    .loc[xmm_desi.groupby('IAUNAME')['desi_sep'].idxmin()]
    )

xmm_hostless = xmm_desi_closest.query("desi_sep>2*xmm_pos_r98 & desi_sep>2*1.43")
xmm_hostless = xmm_hostless.query('Separation<20')

print('hostless xmm near chandra sources', xmm_hostless.shape[0])
xmm_hostless = xmm_hostless[['srcname_fin',	'RA_fin',	'DEC_fin', 'pos_r98', 'flux_05-20', 'IAUNAME', 'SC_RA',	'SC_DEC', 'xmm_pos_r98', 'flux_05_2', 'Separation', 'desi_desi_id', 'desi_ra', 'desi_dec', 'desi_sep', 'xmm_ero_flux_ratio']]

xmm_hostless.rename(columns = {'Separation':'ero_xmm_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

xmm_hostless = add_separation_columns(xmm_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.IAUNAME.nunique()
assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.shape[0]
xmm_hostless.sample(7)

hostless xmm near chandra sources 32


,srcname_fin,RA_fin,DEC_fin,pos_r98,flux_05-20,IAUNAME,SC_RA,SC_DEC,xmm_pos_r98,flux_05_2,ero_xmm_sep,desi_desi_id,desi_ra,desi_dec,desi_sep_minimal,xmm_ero_flux_ratio,desi_ero_sep
2694,SRGe J105506.3+572958,163.776406,57.499494,10.986148,3.807631e-15,4XMM J105506.1+573000,163.775791,57.500237,1.296853,5.382030e-15,2.928893,9011_609951_3424,163.774108,57.502830,9.884814,1.413485,12.805953
2281,SRGe J104138.7+574650,160.411262,57.780564,8.295113,5.042960e-15,4XMM J104138.3+574651,160.409876,57.780996,2.306739,4.028910e-15,3.079815,9011_610719_3295,160.412722,57.783042,9.169144,0.798918,9.347914
2032,SRGe J103847.3+575558,159.697292,57.932746,6.254682,6.003285e-15,4XMM J103847.4+575558,159.697649,57.933018,2.025922,2.550572e-14,1.194767,9011_611487_3072,159.695334,57.933837,5.318569,4.248627,5.427067
2550,SRGe J104122.8+590252,160.344824,59.047725,8.902710,4.231633e-15,4XMM J104122.6+590255,160.344377,59.048800,3.592702,2.114995e-15,3.958391,9011_614514_966,160.344296,59.051217,8.701604,0.499806,12.609070
1027,SRGe J103748.7+585641,159.452829,58.944819,5.360453,1.185179e-14,4XMM J103748.4+585642,159.451735,58.945107,2.638614,1.396732e-14,2.280731,9011_614512_1574,159.448637,58.946069,6.713457,1.178499,8.990060
2088,SRGe J105228.9+574333,163.120323,57.725792,6.977685,5.856936e-15,4XMM J105229.0+574331,163.121200,57.725425,2.956029,6.243270e-15,2.140836,9011_610725_3071,163.116521,57.726276,9.502710,1.065962,7.515293
2153,SRGe J103349.3+584441,158.455573,58.744828,6.972946,5.515691e-15,4XMM J103349.2+584440,158.455253,58.744687,3.907439,9.506640e-15,0.782359,9011_613762_405,158.448776,58.745133,12.203850,1.723563,12.743523


# Joining CSC and XMM hostless candidates

In [44]:
csc_ctps = csc_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'pos_r98', 'flux_05-20', 'ero_csc_sep', 'csc_name',   'csc_ra', 'csc_dec', 'csc_flux_05_2', 'csc_r_98_csc', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'desi_ero_sep', 'csc_ero_flux_ratio']]

xmm_ctps = xmm_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'pos_r98', 'flux_05-20', 'ero_xmm_sep','IAUNAME', 'SC_RA','SC_DEC',  'flux_05_2', 'xmm_pos_r98', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'desi_ero_sep',  'xmm_ero_flux_ratio']]

csc_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)
xmm_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)

In [45]:
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] <  5 ]
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] >  1/5 ]

csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] <  5 ]
csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] >  1/5 ]

In [46]:
xmm_ctps['x_ray_det'] = 'xmm'
csc_ctps['x_ray_det'] = 'csc'
print('XMM companions', xmm_ctps.shape[0])
print('CSC companions', csc_ctps.shape[0])
print('Total companions possible', xmm_ctps.shape[0] + csc_ctps.shape[0])

XMM companions 31
CSC companions 21
Total companions possible 52


In [47]:
final_ctps = pd.concat([xmm_ctps, csc_ctps])
final_ctps.sort_values(by='srcname_fin', inplace=True)
final_ctps.reset_index(drop=True, inplace=True)


n_ctps = final_ctps.groupby('srcname_fin')['x_ray_det'].transform(lambda x: len(x.unique())) 
final_ctps['n_x_ray_det'] = n_ctps


n_desi_ctps = final_ctps.groupby('srcname_fin')['desi_id'].transform(lambda x: len(x.unique()))
final_ctps['n_desi_ctps'] = n_desi_ctps


final_ctps = final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )')
## final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )') - opposite of this


final_ctps.drop_duplicates(subset=['srcname_fin', 'desi_id'], inplace=True)
print('Final number of hostless', final_ctps.shape[0])
final_ctps.reset_index()



Final number of hostless 49


,index,srcname_fin,RA_fin,DEC_fin,pos_r98,flux_05-20,ero_xmm_sep,IAUNAME,SC_RA,SC_DEC,flux_05_2,xmm_pos_r98,desi_sep_minimal,desi_id,desi_ra,desi_dec,desi_ero_sep,xmm_ero_flux_ratio,x_ray_det,ero_csc_sep,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_r_98_csc,csc_ero_flux_ratio,n_x_ray_det,n_desi_ctps
0,0,SRGe J103121.9+573134,157.841091,57.526169,8.679921,4.350933e-15,NaN,NaN,NaN,NaN,NaN,NaN,8.192553,9011_609939_482,157.839339,57.524106,8.162275,NaN,csc,0.504317,2CXO J103121.9+573134,157.841331,57.526115,3.862979e-15,2.331750,0.887851,1,1
1,1,SRGe J103158.3+573841,157.993098,57.644831,11.907729,3.019065e-15,NaN,NaN,NaN,NaN,NaN,NaN,8.059029,9011_610714_3345,157.981929,57.647060,22.964569,NaN,csc,14.930985,2CXO J103156.5+573846,157.985747,57.646145,1.189964e-14,1.155268,3.941499,1,1
2,2,SRGe J103220.4+573211,158.084926,57.536450,5.714911,9.351694e-15,NaN,NaN,NaN,NaN,NaN,NaN,9.735829,9011_609939_2890,158.088107,57.538123,8.606599,NaN,csc,1.421180,2CXO J103220.2+573211,158.084192,57.536421,4.852860e-15,1.498974,0.518928,1,1
3,3,SRGe J103239.4+574033,158.163965,57.675913,6.556224,8.782485e-15,NaN,NaN,NaN,NaN,NaN,NaN,9.702210,9011_610715_599,158.169132,57.676603,10.251895,NaN,csc,2.364492,2CXO J103239.3+574035,158.164092,57.676566,8.115216e-15,1.199912,0.924023,2,1
4,5,SRGe J103251.8+574550,158.215643,57.763855,11.176683,2.601944e-15,4.011813,4XMM J103252.0+574546,158.216941,57.762982,2.699190e-15,1.462532,8.608715,9011_610715_1014,158.221411,57.763161,11.355228,1.037374,xmm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
5,7,SRGe J103302.4+580241,158.260049,58.044662,6.663467,6.085609e-15,NaN,NaN,NaN,NaN,NaN,NaN,6.516825,9011_611484_2685,158.264079,58.045970,9.006889,NaN,csc,3.061539,2CXO J103302.7+580240,158.261655,58.044693,5.264955e-15,2.090497,0.865148,1,1
6,8,SRGe J103349.3+584441,158.455573,58.744828,6.972946,5.515691e-15,0.782359,4XMM J103349.2+584440,158.455253,58.744687,9.506640e-15,3.907439,12.203850,9011_613762_405,158.448776,58.745133,12.743523,1.723563,xmm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
7,9,SRGe J103409.4+574725,158.539363,57.790228,7.180546,4.495352e-15,NaN,NaN,NaN,NaN,NaN,NaN,6.808410,9011_610715_3565,158.543036,57.792069,9.673252,NaN,csc,3.623997,2CXO J103409.5+574728,158.539889,57.791195,2.186806e-15,1.230499,0.486459,1,1
8,10,SRGe J103518.6+562405,158.827436,56.401399,6.233959,8.711545e-15,5.638750,4XMM J103518.0+562408,158.825121,56.402300,9.770200e-15,3.030869,6.995479,9011_606788_302,158.827919,56.401126,1.375083,1.121523,xmm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
9,11,SRGe J103520.2+573355,158.834253,57.565203,8.727764,3.591921e-15,NaN,NaN,NaN,NaN,NaN,NaN,6.015550,9011_609941_914,158.835675,57.562204,11.139738,NaN,csc,7.821427,2CXO J103520.9+573349,158.837140,57.563679,4.241985e-15,2.384348,1.180980,1,1


In [48]:
# #save final_ctps_csv to csv
final_ctps_csv_coords = final_ctps[['srcname_fin', 'RA_fin',	'DEC_fin',	'pos_r98']]
final_ctps_csv_coords_matched = cross_match_data_frames(final_ctps_csv_coords, desi[['desi_id', 'ra', 'dec', 'type']], 'RA_fin', 'DEC_fin', 'ra', 'dec', closest=True, match_radius=30)
final_ctps_csv_coords_matched

cross-match radius 30 arcsec
total matches: 622 out of 49 x 2418574
	 total unique pairs: 622
	 total non-unique pairs (duplicates in df2): 0
total closest matches: 49


,srcname_fin,RA_fin,DEC_fin,pos_r98,desi_id,ra,dec,type,sep,n_near,n_matches
12,SRGe J103121.9+573134,157.841091,57.526169,8.679921,9011_609939_482,157.839339,57.524106,REX,8.162275,17,1
25,SRGe J103158.3+573841,157.993098,57.644831,11.907729,9011_610714_3400,157.988800,57.643752,PSF,9.144548,12,1
30,SRGe J103220.4+573211,158.084926,57.536450,5.714911,9011_609939_2890,158.088107,57.538123,PSF,8.606599,12,1
43,SRGe J103239.4+574033,158.163965,57.675913,6.556224,9011_610715_599,158.169132,57.676603,DEV,10.251895,16,1
61,SRGe J103251.8+574550,158.215643,57.763855,11.176683,9011_610715_1014,158.221411,57.763161,REX,11.355228,12,1
72,SRGe J103302.4+580241,158.260049,58.044662,6.663467,9011_611484_2685,158.264079,58.045970,REX,9.006889,5,1
81,SRGe J103349.3+584441,158.455573,58.744828,6.972946,9011_613762_405,158.448776,58.745133,PSF,12.743523,11,1
97,SRGe J103409.4+574725,158.539363,57.790228,7.180546,9011_610715_3500,158.536213,57.788361,REX,9.038309,15,1
105,SRGe J103518.6+562405,158.827436,56.401399,6.233959,9011_606788_302,158.827919,56.401126,REX,1.375083,18,1
121,SRGe J103520.2+573355,158.834253,57.565203,8.727764,9011_609941_904,158.834278,57.566603,REX,5.041880,15,1


In [49]:
final_ctps_csv_coords_matched.query('sep>pos_r98 & type=="PSF"')

,srcname_fin,RA_fin,DEC_fin,pos_r98,desi_id,ra,dec,type,sep,n_near,n_matches
30,SRGe J103220.4+573211,158.084926,57.536450,5.714911,9011_609939_2890,158.088107,57.538123,PSF,8.606599,12,1
81,SRGe J103349.3+584441,158.455573,58.744828,6.972946,9011_613762_405,158.448776,58.745133,PSF,12.743523,11,1
137,SRGe J103525.8+595331,158.857628,59.891915,8.250369,9011_617451_1827,158.861910,59.888438,PSF,14.712714,8,1
165,SRGe J103748.7+585641,159.452829,58.944819,5.360453,9011_614512_1573,159.448476,58.943895,PSF,8.741244,22,1
232,SRGe J104122.8+590252,160.344824,59.047725,8.902710,9011_614514_927,160.339396,59.048743,PSF,10.696558,15,1
269,SRGe J104341.0+590023,160.920944,59.006447,10.333953,9011_614515_1534,160.919953,59.009420,PSF,10.858004,14,1
290,SRGe J104453.1+585450,161.221287,58.913756,8.352394,9011_614516_382,161.226910,58.914631,PSF,10.915065,8,1
384,SRGe J104656.6+572859,161.735777,57.482989,5.116249,9011_609947_2233,161.739150,57.482406,PSF,6.854900,12,1
600,SRGe J105640.4+573203,164.168282,57.534091,6.063802,9011_609952_2647,164.167686,57.536689,PSF,9.422389,14,1


In [50]:
# #save final_ctps_csv to csv
final_ctps_csv = final_ctps[['srcname_fin', 'desi_id']]
final_ctps_csv.rename(columns={'srcname_fin':'ID'}, inplace=True)
final_ctps_csv.rename(columns={'desi_id':'desi_id_true_ctp'}, inplace=True)
final_ctps_csv.desi_id_true_ctp = 'hostless'
final_ctps_csv.ID = final_ctps_csv.ID.str.encode('utf-8')
final_ctps_csv.desi_id_true_ctp = final_ctps_csv.desi_id_true_ctp.str.encode('utf-8')
final_ctps_csv.to_pickle(data_path+'validation_ctps_ero_desi_lh_hostless.pkl')
final_ctps_csv

,ID,desi_id_true_ctp
0,b'SRGe J103121.9+573134',b'hostless'
1,b'SRGe J103158.3+573841',b'hostless'
2,b'SRGe J103220.4+573211',b'hostless'
3,b'SRGe J103239.4+574033',b'hostless'
5,b'SRGe J103251.8+574550',b'hostless'
7,b'SRGe J103302.4+580241',b'hostless'
8,b'SRGe J103349.3+584441',b'hostless'
9,b'SRGe J103409.4+574725',b'hostless'
10,b'SRGe J103518.6+562405',b'hostless'
11,b'SRGe J103520.2+573355',b'hostless'


# Combine non-hostless and hostless catalogs to produce final validation sample

In [51]:
df_ctps = pd.read_pickle(data_path+'validation_ctps_ero_desi_lh_no_hostless.pkl')
df_hostless = pd.read_pickle(data_path+'validation_ctps_ero_desi_lh_hostless.pkl')
df_ctps = pd.concat([df_ctps, df_hostless])

df_ctps.to_pickle(data_path+'validation_ctps_ero_desi_lh.pkl')
df_ctps.sample(10)

,ID,desi_id_true_ctp
56,b'SRGe J103327.6+574901',b'9011_610715_2134'
31,b'SRGe J103244.2+595710',b'9011_617450_468'
383,b'SRGe J104658.2+572749',b'9011_609947_2276'
93,b'SRGe J103409.6+584921',b'9011_613762_1073'
615,b'SRGe J105453.8+574020',b'9011_610727_283'
530,b'SRGe J105039.5+572335',b'9011_609949_1963'
336,b'SRGe J104539.8+590131',b'9011_614516_1732'
162,b'SRGe J103557.2+580636',b'9011_611486_1294'
505,b'SRGe J104954.5+592312',b'9011_615999_1133'
193,b'SRGe J103907.5+574921',b'9011_610718_2416'
